In [57]:
import requests
import time
import hashlib
import hmac
import base64
import csv
import os

# ✅ API Credentials (Replace with actual API keys)
API_KEY = "09f5e1df-1809-4b19-92c9-a9c797c7f111"
API_SECRET = "P8dTv/xKgfQhRymutcnrt8TbpsbhJQu"

# ✅ Generate API Signature (For Authentication)
def generate_signature(api_secret, timestamp):
    """
    Generates a secure HMAC SHA256 signature using the API secret key.
    Fixes 'Incorrect padding' error by ensuring the API secret is correctly base64-encoded.
    """
    # Ensure API secret is properly base64-encoded
    try:
        missing_padding = len(api_secret) % 4
        if missing_padding:
            api_secret += '=' * (4 - missing_padding)  # Fix padding issue

        secret = base64.b64decode(api_secret)  # Decode safely
    except Exception as e:
        raise ValueError(f"Invalid API Secret: {e}")  # Handle decoding errors

    # Generate the signature
    message = timestamp.encode('utf-8')
    signature = hmac.new(secret, message, hashlib.sha256).digest()
    return base64.b64encode(signature).decode()

# ✅ MODULE 1: Fetch Market Data (Live Ticker Data)
def get_market_data():
    """
    Fetches the latest market price, 24-hour average price, and trading volume.
    Helps in making trading decisions.
    """
    url = "https://api.btcturk.com/api/v2/ticker"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if "data" in data:
            for ticker in data["data"]:
                print(f"Pair: {ticker['pair']} | Last Price: {ticker['last']} | Volume: {ticker['volume']}")
                print("-" * 30)
        else:
            print("Unexpected response structure.")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching market data: {e}")

# ✅ MODULE 2: Fetch Historical Data (For Strategy Analysis)
def get_historical_data(pair_symbol, interval):
    """
    Fetches historical price data (OHLC) for a given trading pair.
    Useful for analyzing price trends before placing an order.
    """
    url = "https://api.btcturk.com/api/v2/ohlc"
    params = {"pairSymbol": pair_symbol, "interval": interval}
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if "data" in data:
            for entry in data["data"]:
                print(f"Time: {entry['time']} | Open: {entry['open']} | Close: {entry['close']}")
        else:
            print("Unexpected response structure.")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching historical data: {e}")

# ✅ MODULE 3: Place an Order (Buy/Sell Trade Execution)
def place_order(pair_symbol, quantity, price, order_method, order_type):
    """
    Places a buy or sell order on the exchange.
    Logs the order attempt to CSV for tracking.
    """
    url = "https://api.btcturk.com/api/v1/order"
    timestamp = str(int(time.time() * 1000))
    signature = generate_signature(API_SECRET, timestamp)

    headers = {
        "X-PCK": API_KEY,
        "X-Stamp": timestamp,
        "X-Signature": signature,
        "Content-Type": "application/json"
    }

    payload = {
        "pairSymbol": pair_symbol,
        "quantity": quantity,
        "price": price,
        "orderMethod": order_method,  # "Limit" or "Market"
        "orderType": order_type  # "Buy" or "Sell"
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        data = response.json()
        print("✅ Order Placed Successfully:", data)
        log_trade(pair_symbol, quantity, price, order_method, order_type, "Success", "Order placed successfully")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error placing order: {e}")
        log_trade(pair_symbol, quantity, price, order_method, order_type, "Failed", str(e))

# ✅ MODULE 4: Track Live Trade Status (Monitor Open & Closed Orders)
def track_live_trades():
    """
    Fetches live trade status from the exchange.
    Checks the status of open and closed orders in real-time.
    """
    url = "https://api.btcturk.com/api/v1/openOrders"
    timestamp = str(int(time.time() * 1000))
    signature = generate_signature(API_SECRET, timestamp)

    headers = {
        "X-PCK": API_KEY,
        "X-Stamp": timestamp,
        "X-Signature": signature,
        "Content-Type": "application/json"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        orders = response.json()

        print("\n🔍 Live Trade Status:")
        for order in orders.get("data", []):
            print(f"Order ID: {order['id']} | Pair: {order['pairSymbol']} | Price: {order['price']} | Status: {order['status']}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error tracking live trades: {e}")

# ✅ MODULE 5: Log Trades (Save Order Data to CSV)
def log_trade(pair, quantity, price, method, order_type, status, reason):
    """
    Saves all trade attempts (success or failure) to a CSV file.
    Helps in maintaining a trade history for analysis.
    """
    file_exists = os.path.exists("trade_history.csv")
    
    with open("trade_history.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Timestamp", "Pair", "Quantity", "Price", "Method", "Order Type", "Status", "Reason"])
        writer.writerow([time.strftime('%Y-%m-%d %H:%M:%S'), pair, quantity, price, method, order_type, status, reason])

    print("📁 Trade logged successfully.")

# ✅ MAIN FUNCTION: Runs All Modules
if __name__ == "__main__":
    get_market_data()  # Module 1: Fetch live market data
    get_historical_data("BTCUSDT", "15m")  # Module 2: Fetch historical price data
    place_order("BTCUSDT", 0.01, 50000, "Limit", "Buy")  # Module 3: Place a sample order
    track_live_trades()  # Module 4: Monitor live trades


Pair: BTCTRY | Last Price: 3161132 | Volume: 68.11646754
------------------------------
Pair: ETHTRY | Last Price: 79858 | Volume: 1494.11770196
------------------------------
Pair: XRPTRY | Last Price: 86.1 | Volume: 5048758.5122
------------------------------
Pair: LTCTRY | Last Price: 3790.67 | Volume: 4240.07159373
------------------------------
Pair: USDTTRY | Last Price: 36.579 | Volume: 14274296.35
------------------------------
Pair: BTCUSDT | Last Price: 86401 | Volume: 52.15147105
------------------------------
Pair: ETHUSDT | Last Price: 2182.0 | Volume: 1026.37523082
------------------------------
Pair: XRPUSDT | Last Price: 2.3521 | Volume: 2696628.283
------------------------------
Pair: LTCUSDT | Last Price: 103.8 | Volume: 3098.13324462
------------------------------
Pair: XLMTRY | Last Price: 10.149 | Volume: 761804.3384
------------------------------
Pair: XLMUSDT | Last Price: 0.2758 | Volume: 164609.9301
------------------------------
Pair: NEOTRY | Last Price: 321.